In [1]:
import os 
import warnings 
warnings.filterwarnings('ignore')
import cv2 
import pandas as pd 
import numpy as np 
from glob import glob 
from tqdm import tqdm 
import random 

import torch 
import torch.nn as nn 
import torchvision 
import torchvision.transforms as transforms 

import utils
from utils.Dataset import Data_loader,train_valid_split 
from utils import Model 
from sklearn.metrics import f1_score

# 데이터 로드 

In [4]:
img_dirs = sorted(glob('./Data/test/*.jpg'))
df = pd.read_csv('./Data/test.csv')

class cfg:
    img_size = 224 
    batch_size = 16
    train_ratio = 0.8 
    num_epochs = 50 
    num_fold = 1 
    model_name = 'resnet50'
    lr = 1e-3 
    device = 'cuda:0'
    sava_path = './Save_models/'

def Data_init():
    train_csv = pd.read_csv('./Data/train.csv')
    img_dirs = np.array(sorted(glob('./Data/test/*.jpg')))
    return img_dirs,train_csv

img_dirs,train_csv = Data_init()
label_decoder = {key:value for key,value in enumerate(np.unique(train_csv['artist']))}

augmenter = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(size = (cfg.img_size,cfg.img_size)),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    ])

data = Data_loader(img_dirs,np.zeros(len(img_dirs)),cfg,augmenter=augmenter,shuffle=False)
model_name = 'skfold_effb4/4_fold_best.pt'
model = torch.load(f'./Save_models/{model_name}').to(cfg.device)
model.eval()

f_pred = [] 
with torch.no_grad():
    for batch in (data):
        x = torch.tensor(batch[0]).type(torch.float32).to(cfg.device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())
f_result = [label_decoder[result] for result in f_pred ]


submission = pd.read_csv('./Data/sample_submission.csv')
submission['artist'] = f_result 
submission.to_csv("./submission/skfold_effb4.csv", index = False)